In [148]:
#WebScraping Realtime stock prices from YAHOO Finance and Loading that data into SQL table

In [183]:
import pandas as pd

# Importing the StringIO module.
from io import StringIO

# import the requests library 
import requests

In [184]:
del[base_df]
base_df = pd.DataFrame()

In [185]:
#Base Url to fetch historical data through request
#List holds various stocks
lst = ['MRF','SBIN','TCS','HCLTECH','ROSSARI','PIDILITIND']
start_date= 946684800 #Jan 1 2000 in Unix format
end_date= 1714003200 #Apr 24 2024 in Unix format

#Defining headers with User agent to establish requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest'
}


In [181]:
#Function stock_price scrape real-time data
#Scraping Real-time Stock data from YAHOO Finance for stocks
#Iterating through lists to dynamically create respective url for stocks
def yahoo_scrape_stock_price_data(lst,start_date,end_date):
    
    #Creating basedataframe to hold complete data
    base_df = pd.DataFrame()
    
    for x in range(len(lst)):
        print(x, lst[x])
        url = "https://query1.finance.yahoo.com/v7/finance/download/{}.NS?period1={}&period2={}&interval=1d&events=history&includeAdjustedClose=true".format(lst[x],start_date,end_date)
        #print(url)
        response = requests.get(url = url, headers = headers)
        df = pd.read_csv(StringIO(response.content.decode('utf-8')))
        
        #Rephrasing / Adding dataframe by including new column with script name
        df.insert(loc=0,column='Stock',value=lst[x])
        
        #Appending the results to the base Dataframe for consolidated view
        base_df = base_df._append(df,ignore_index=True)
        #df.append(df)
        #df.columns
        print(lst[x],response.content)
        #Formating datatypes on Base Dataframe to its appropriate format


    #Formating datatypes on Base Dataframe to appropriate format for further SQL data load process

    #Changing Object type to Date format
    base_df['Date'] = pd.to_datetime(base_df['Date'])

    #Formatting Volume to Integer from Float type to make it easier
    base_df.Volume = base_df.Volume.astype('Int64')

    #Rounding the prices to two decimals to multiple columns
    base_df[['Open','High','Low','Close','Adj Close']] = base_df[['Open','High','Low','Close','Adj Close']].round(decimals=2)

    #Changing column name on base Data frame to appropriate format
    base_df.rename(columns={'Adj Close': 'Adj_Close'},inplace= True)

    #Changing all column names to lowercase 
    base_df.columns = map(str.lower,base_df.columns)

    #Removing records which has volume as 0 because it holds discrepency data
    base_df.drop(base_df[base_df.volume==0].index,inplace= True)
    
    #Inserting stock data into SQL database using sqlachemy
    from sqlalchemy import create_engine

    table_name = 'stock_ohlc_data'

    sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')

    db_conn= sql_engine.connect()

    #Defining table name on MSSQL server to locate the data
    table_name = 'stock_ohlc_data'

    #DB Actions to load data from Pandas Dataframe to MSSQL
    try:
        base_df.to_sql(table_name, db_conn, if_exists= 'replace',index= False)
    except Exception as ex:
        print(ex)
    else:
        print('Stock OHLC price details data successfully inserted into MS SQL table-{}'.format(table_name))
    finally:
        db_conn.close()

    return base_df

In [186]:
#Invoking Functions to call actions for Scraping real-time data and data-cleaning processs
yahoo_scrape_stock_price_data(lst,start_date,end_date)


0 MRF
MRF b'Date,Open,High,Low,Close,Adj Close,Volume\n2002-07-01,910.000000,910.000000,880.000000,897.599976,813.425842,426\n2002-07-02,900.000000,905.000000,895.000000,903.650024,818.908386,310\n2002-07-03,910.000000,910.000000,900.000000,906.700012,821.672424,336\n2002-07-04,920.000000,925.000000,910.000000,916.799988,830.825134,591\n2002-07-05,915.000000,935.000000,910.000000,927.299988,840.340820,1623\n2002-07-08,930.000000,933.000000,918.000000,929.049988,841.926575,1595\n2002-07-09,934.000000,934.000000,900.150024,904.400024,819.588135,586\n2002-07-10,910.000000,917.900024,901.000000,910.700012,825.297424,826\n2002-07-11,891.000000,920.000000,885.000000,893.950012,810.118164,1265\n2002-07-12,908.900024,909.000000,890.099976,896.500000,812.428772,361\n2002-07-15,900.000000,910.000000,871.200012,875.000000,792.945068,1265\n2002-07-16,890.000000,890.000000,880.000000,883.049988,800.240417,552\n2002-07-17,880.099976,883.000000,880.000000,881.650024,798.971741,572\n2002-07-18,867.000

,stock,date,open,high,low,close,adj_close,volume
0,MRF,2002-07-01,910.00,910.00,880.00,897.60,813.43,426
1,MRF,2002-07-02,900.00,905.00,895.00,903.65,818.91,310
2,MRF,2002-07-03,910.00,910.00,900.00,906.70,821.67,336
3,MRF,2002-07-04,920.00,925.00,910.00,916.80,830.83,591
4,MRF,2002-07-05,915.00,935.00,910.00,927.30,840.34,1623
...,...,...,...,...,...,...,...,...
28645,PIDILITIND,2024-04-18,2887.45,2921.75,2860.55,2871.80,2871.80,328125
28646,PIDILITIND,2024-04-19,2859.05,2860.00,2804.60,2830.70,2830.70,668800
28647,PIDILITIND,2024-04-22,2837.05,2911.00,2835.05,2898.05,2898.05,227565
28648,PIDILITIND,2024-04-23,2919.85,2925.00,2875.55,2914.40,2914.40,379694


""
